In [4]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col, count, when
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer, OneHotEncoder
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType


def categorize_columns(df):
  numerical_cols = []
  categorical_cols = []

  for column, dtype in df.dtypes:
    if dtype in ['double', 'int']:
      numerical_cols.append(column)
    else:
      categorical_cols.append(column)

  return numerical_cols, categorical_cols

spark = SparkSession.builder.appName('ChurnPrediction').getOrCreate()
df = spark.read.option('header', 'true').csv('../data/raw/data.csv', inferSchema=True)
df = df.drop('customerID')
df = df.withColumn("TotalCharges", col("TotalCharges").cast("double"))
numerical_cols, categorical_cols = categorize_columns(df)
df = df.dropDuplicates()
df = df.na.drop()

numerical_features_df = VectorAssembler(
    inputCols=numerical_cols, 
    outputCol="features"
).transform(df)
standardScaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
scaled_numerical_df = standardScaler.fit(numerical_features_df).transform(numerical_features_df)
inx_categorical_col = [c + "_inx" for c in categorical_cols]
indexer = StringIndexer(inputCols=categorical_cols, outputCols=inx_categorical_col)
inx_df = indexer.fit(scaled_numerical_df).transform(scaled_numerical_df)
inx_categorical_col = inx_categorical_col[:-1]
categorical_features_df = VectorAssembler(
    inputCols=inx_categorical_col, 
    outputCol="cat_features_inx"
).transform(inx_df)
final_df = VectorAssembler(
    inputCols=["scaledFeatures", "cat_features_inx"], 
    outputCol="final_feature_vector"
).transform(categorical_features_df)
input_df = final_df.select("final_feature_vector","Churn_inx")

major_df = input_df.filter(col('Churn_inx') == 0)
minor_df = input_df.filter(col('Churn_inx') == 1)
major_count = major_df.count()
minor_count = minor_df.count()
ratio = major_count / minor_count
oversampled_minor_df = minor_df.sample(withReplacement=True, fraction=ratio)
input_data = major_df.unionAll(oversampled_minor_df)


def vector_to_string(vector):
    return str(vector)

# Buat UDF menggunakan fungsi di atas
vector_to_string_udf = udf(vector_to_string, StringType())
# Terapkan UDF ke kolom vektor untuk membuat kolom baru yang berisi representasi string dari vektor
df_with_vector_as_string = input_data.withColumn("vector_as_string", vector_to_string_udf("final_feature_vector"))
# Specify the path where you want to save the CSV file and write the DataFrame to CSV
df_with_vector_as_string.select("vector_as_string", "Churn_inx").write.csv(
    "../data/processed/output_data.csv", 
    header=True, 
    mode="overwrite"
)


Py4JJavaError: An error occurred while calling o740.csv.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java: